In [1]:
!pip install pymongo

# Connecting to mongodb


In [1]:
# importing the required libraries
import pymongo
import pprint
import json
import warnings
import pandas as pd, numpy as np
warnings.filterwarnings('ignore')

# connect to the mongoclient
client = pymongo.MongoClient('mongodb://localhost:27017')

# get the database
database = client['intern']

In [2]:
sensor_data = database.get_collection("sensor_data")

In [3]:
column_names = sensor_data.distinct("sensor_name")

### Single feature analysis for data from mongo end

In [5]:
motor_voltage = sensor_data.find({"sensor_name":'Motor Voltage'}, {"data":1})
for doc in motor_voltage:
    print(len(doc['data']))

5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760
5760


#### each unit of list in data has two set of variables one is the timestamp and the other is that specifc sensot values

## Initial analysis to check how to combine columns

In [9]:
cursor = sensor_data.find({"sensor_name":'Motor Voltage'}, 
                                    {"data":1})

pd_df = pd.DataFrame(columns=["timestamp",'value'])

for doc in cursor:
    temp_df = pd.DataFrame(doc['data'], columns=["timestamp",'value'])
    pd_df = pd.concat([pd_df, temp_df])

pd_df.sort_values(by='timestamp').reset_index(drop=True)

,timestamp,value
0,2018-01-01 00:00:15,4234.068848
1,2018-01-01 00:00:30,4235.0
2,2018-01-01 00:00:45,4236.45459
3,2018-01-01 00:01:00,4232.772461
4,2018-01-01 00:01:15,4240.088379
...,...,...
4204794,2019-12-31 23:58:45,4238.430176
4204795,2019-12-31 23:59:00,4232.068848
4204796,2019-12-31 23:59:15,4240.943848
4204797,2019-12-31 23:59:30,4235.980957


In [11]:
del pd_df
gc.collect()

226

In [6]:
pd.DataFrame(sensor_data.find_one({"sensor_name":'Motor Voltage'}, 
                                    {"data":1})["data"], 
                                    columns=["timestamp",'value']).merge(pd.DataFrame(sensor_data.find_one({"sensor_name":'Pump Shaft Speed'}, {"data":1})["data"], 
                                                                                        columns=['timestamp','value']), on='timestamp')

,timestamp,value_x,value_y
0,2018-01-01 00:00:15,4234.068848,3589.281494
1,2018-01-01 00:00:30,4235.000000,3589.281982
2,2018-01-01 00:00:45,4236.454590,3589.282715
3,2018-01-01 00:01:00,4232.772461,3589.283203
4,2018-01-01 00:01:15,4240.088379,3589.283691
...,...,...,...
5755,2018-01-01 23:59:00,4233.307617,3589.990234
5756,2018-01-01 23:59:15,4242.801270,3589.990234
5757,2018-01-01 23:59:30,4229.829590,3589.990234
5758,2018-01-01 23:59:45,4234.789551,3589.990234


All the 5760 rows were displayed after merging two features, so no data loss is observed.

Columns names have to be changed to what they represent, some tranformation is required.

## Final data creation for modelling

### Extract timestamp as a single dataframe to merge to other columns

In [12]:
cursor = sensor_data.find({"sensor_name":'Motor Voltage'}, 
                                    {"data":1})

pd_df = pd.DataFrame(columns=["timestamp"])

for doc in cursor:
    temp_df = pd.DataFrame(doc['data'], columns=["timestamp",'value'])[['timestamp']]
    pd_df = pd.concat([pd_df, temp_df])

pd_df.sort_values(by='timestamp').reset_index(drop=True)

pd_df.head()

,timestamp
0,2018-01-01 00:00:15
1,2018-01-01 00:00:30
2,2018-01-01 00:00:45
3,2018-01-01 00:01:00
4,2018-01-01 00:01:15


In [13]:
pd_df.shape

(4204799, 1)

### Created a single column df with just timestamp so that other columns can be merged to it one by one and we don't have to fruther clean it

In [34]:
len(column_names)

26

### As number of columns are only 26 no loop opimization is required, instead of using map or vectorization on python lists, as simple for loop will do ther trick

In [14]:
for col in column_names:
    cursor = sensor_data.find({"sensor_name":col}, 
                                    {"data":1})

    col_df = pd.DataFrame(columns=["timestamp",'value'])

    for doc in cursor:
        temp_df = pd.DataFrame(doc['data'], columns=["timestamp",'value'])
        col_df = pd.concat([col_df, temp_df])

    col_df = col_df.sort_values(by='timestamp').reset_index(drop=True).rename(columns ={"value":col})

    
                                    
    pd_df = pd_df.merge(col_df, on='timestamp')

pd_df

,timestamp,Auxiliary Boilers A/B Feed Water Header Pressure 2,Auxiliary Boilers Feed Water Header Pressure,Heat Recovery System Header Mass Flow,Heat Recovery System Header Mass Flow.1,Heat Recovery System Header Pressure,Heat Recovery System Header Pressure.1,Lube Oil Cooler Outlet Temperature,Lube Oil Tank Temperature,Motor Current Phase A,...,Pump Journal 1 Bearing Temperature,Pump Journal 2 Bearing Temperature,Pump Radial Bearing Vibration,Pump Shaft Speed,Pump Suction Pressure 1,Pump Suction Pressure 2,Pump Suction Strainer Differential Pressure,Pump Suction Temperature,Pump Thrust Bearing Temperature 1,Pump Thrust Bearing Temperature 2
0,2018-01-01 00:00:15,1305.491089,1305.893799,301258.6563,304479.0625,1779.438599,1782.647461,71.223015,76.965309,145.308487,...,137.135651,144.30011,9.661202,3589.281494,24.95101,16.886732,0.27272,226.743881,98.050789,86.571556
1,2018-01-01 00:00:30,1301.227783,1299.925171,303027.75,303782.2813,1779.364014,1782.624756,71.215706,76.948776,145.0,...,137.133484,144.299057,9.655985,3589.281982,25.042177,16.886147,0.272987,226.743118,98.054756,86.565018
2,2018-01-01 00:00:45,1296.178589,1294.646606,301878.9688,303518.4375,1779.289551,1782.602173,71.208389,76.932243,146.306839,...,137.131317,144.298004,9.650768,3589.282715,25.012571,16.885561,0.273254,226.742371,98.058716,86.558487
3,2018-01-01 00:01:00,1297.628296,1299.98877,302023.0625,303576.375,1779.214966,1782.579468,71.20108,76.912827,146.0,...,137.12915,144.296936,9.645551,3589.283203,25.00959,16.884975,0.273521,226.741623,98.062683,86.551956
4,2018-01-01 00:01:15,1299.078003,1300.749023,301972.0625,303999.5313,1779.140381,1782.556885,71.193764,76.89296,146.305389,...,137.126968,144.295883,9.642216,3589.283691,25.006611,16.884392,0.273788,226.74086,98.06665,86.545425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204794,2019-12-31 23:58:45,1305.224976,1305.452515,325015.4688,330229.3438,1702.869385,1706.575195,62.233841,61.810455,158.0,...,132.747528,139.821579,8.379042,3590.083252,25.555643,17.410847,0.404363,227.793106,90.913559,79.74984
4204795,2019-12-31 23:59:00,1304.164673,1303.207764,324984.625,329936.0625,1703.680542,1707.391357,62.240265,61.803555,157.493698,...,132.74646,139.821533,8.331427,3590.082275,25.587416,17.416965,0.396943,227.794098,90.91407,79.748138
4204796,2019-12-31 23:59:15,1308.058838,1306.166016,324521.25,329425.6875,1704.491699,1708.207642,62.246689,61.796654,157.018768,...,132.745392,139.821487,8.089752,3590.081055,25.64444,17.423084,0.396188,227.79509,90.914574,79.746437
4204797,2019-12-31 23:59:30,1315.320679,1312.660889,324716.9688,330124.75,1705.302856,1709.023926,62.253113,61.789753,157.975784,...,132.744324,139.821457,7.992675,3590.080078,25.580099,17.429203,0.395433,227.796082,90.915085,79.744728


In [15]:
gc.collect()

50

In [16]:
pd_df.to_csv('data/sensor_data.csv')

NameError: name 'pd_df' is not defined